<a href="https://colab.research.google.com/github/edanielacero/SIS-INT-2023-PRAC-3/blob/main/Proyecto_oficial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
class Estado:
    def __init__(self):
        self.tablero = [['.' for _ in range(8)] for _ in range(8)]
        self.tablero[3][3] = self.tablero[4][4] = 'O'  # Fichas blancas
        self.tablero[3][4] = self.tablero[4][3] = 'X'  # Fichas negras

    def mostrar_tablero(self):
        print('  A B C D E F G H')

        for i, fila in enumerate(self.tablero):
            print(f'{i+1} ', end='')

            for casilla in fila:
                print(casilla, end=' ')
            print()
        print()

    def letra_a_numero_columna(self, letra):
        return ord(letra) - 65  # Convierte la letra a un número de columna (0-7)

    def realizar_movimiento(self, fila, columna_letra, jugador):
      fila = fila - 1
      columna = self.letra_a_numero_columna(columna_letra)
      if self.tablero[fila][columna] == ".":
        self.tablero[fila][columna] = jugador
        adversario = 'O' if jugador == 'X' else 'X'
        direcciones = [(0, 1), (1, 0), (0, -1), (-1, 0), (1, 1), (-1, 1), (1, -1), (-1, -1)]

        for dir_f, dir_c in direcciones:
            i, j = fila + dir_f, columna + dir_c
            if not self.esta_dentro_del_tablero(i, j) or self.tablero[i][j] != adversario:
                continue

            i, j = i + dir_f, j + dir_c
            while self.esta_dentro_del_tablero(i, j) and self.tablero[i][j] == adversario:
                i, j = i + dir_f, j + dir_c

            if self.esta_dentro_del_tablero(i, j) and self.tablero[i][j] == jugador:
                while (i != fila or j != columna):
                    i, j = i - dir_f, j - dir_c
                    self.tablero[i][j] = jugador

    def obtener_movimientos_validos(self, jugador):
        movimientos_validos = []

        for i in range(8):
            for j in range(8):
                if self.tablero[i][j] == '.':
                    if self.es_movimiento_valido(i, j, jugador):
                        fila_numero = i + 1  # Sumamos 1 para tener el número de fila (1-8)
                        columna_letra = chr(j + 65)  # Convertimos el índice a letra (A-H)
                        movimientos_validos.append((fila_numero, columna_letra))

        return movimientos_validos

    def es_movimiento_valido(self, fila, columna, jugador):
        if self.tablero[fila][columna] != '.':
            return False

        adversario = 'O' if jugador == 'X' else 'X'
        direcciones = [(0, 1), (1, 0), (0, -1), (-1, 0), (1, 1), (-1, 1), (1, -1), (-1, -1)]

        for dir_f, dir_c in direcciones:
            i, j = fila + dir_f, columna + dir_c
            if self.esta_dentro_del_tablero(i, j) and self.tablero[i][j] == adversario:
                i, j = i + dir_f, j + dir_c
                while self.esta_dentro_del_tablero(i, j) and self.tablero[i][j] == adversario:
                    i, j = i + dir_f, j + dir_c

                if self.esta_dentro_del_tablero(i, j) and self.tablero[i][j] == jugador:
                    return True

        return False

    def esta_dentro_del_tablero(self, fila, columna):
        return 0 <= fila < 8 and 0 <= columna < 8

    def es_juego_terminado(self):
       return self.tablero_lleno()

    def tablero_lleno(self):
      cont = sum(1 for fila in self.tablero for casilla in fila if casilla != ".")
      return cont == 64

    def obtener_ganador(self):
        contador_X = sum(fila.count('X') for fila in self.tablero)
        contador_O = sum(fila.count('O') for fila in self.tablero)

        if contador_X > contador_O:
            return 'X'
        elif contador_O > contador_X:
            return 'O'
        else:
            return 'Empate'

    def utility(self):
      contador_X = sum(fila.count('X') for fila in self.tablero)
      contador_O = sum(fila.count('O') for fila in self.tablero)
      return contador_X - contador_O

In [28]:
est = Estado()
est.mostrar_tablero()
print(est.obtener_movimientos_validos('X'))
est.realizar_movimiento(3, 'D', 'X')
est.mostrar_tablero()
print(est.es_juego_terminado())
est.utility()

  A B C D E F G H
1 . . . . . . . . 
2 . . . . . . . . 
3 . . . . . . . . 
4 . . . O X . . . 
5 . . . X O . . . 
6 . . . . . . . . 
7 . . . . . . . . 
8 . . . . . . . . 

[(3, 'D'), (4, 'C'), (5, 'F'), (6, 'E')]
  A B C D E F G H
1 . . . . . . . . 
2 . . . . . . . . 
3 . . . X . . . . 
4 . . . X X . . . 
5 . . . X O . . . 
6 . . . . . . . . 
7 . . . . . . . . 
8 . . . . . . . . 

False


3

## Min max

In [37]:
import copy

In [32]:
def terminal_test(state):
  return state.es_juego_terminado()

In [36]:
def result(state, action):
  succesor = copy.deepcopy(state)
  succesor.realizar_movimiento(action[0][0], action[0][1], action[1])
  succesor.mostrar_tablero()
  return succesor

In [5]:
def utility(state):
  return state.utility()

In [34]:
def max_value(state):
    if terminal_test(state):
        return utility(state)

    v = float('-inf') # valor pequeño
    actions = state.obtener_movimientos_validos('X')
    print(actions)
    for action in actions:
        print("Estoy en las acciones de max")
        successor = result(state, (action,'X')) # devuelve estado sucesor
        eval_min = min_value(successor) # evaluar el estado
        v = max(v, eval_min)
    return v

def min_value(state):
    if terminal_test(state):
        return utility(state)

    v = float('inf') # valor grande
    actions = state.obtener_movimientos_validos('O')
    print(actions)
    for action in actions:
        print("Estoy en las acciones de min")
        successor = result(state, (action,'O')) # devuelve estado sucesor
        eval_max = max_value(successor) # evaluar el estado
        v = min(v, eval_max)
    return v

def min_max(state, player):
    if player == "MAX":
        v = []
        actions = state.obtener_movimientos_validos('X')
        print()
        for action in actions:
            print("En mix max superior")
            successor = result(state, (action,'X'))
            v.append((min_value(successor), action))
        return max(v, key=lambda x: x[0])[1]

    elif player == "MIN":
        v = []
        actions = state.obtener_movimientos_validos('O')
        for action in actions:
            successor = result(state, (action,'O'))
            v.append((max_value(successor), action))
        return min(v, key=lambda x: x[0])[1]

In [38]:
initial_state = Estado()
min_max(initial_state, "MAX")

Se truncaron las últimas líneas 5000 del resultado de transmisión.
  A B C D E F G H
1 O O O O O X X X 
2 O O O O X X X X 
3 O O O X O O X X 
4 O O X O O X X X 
5 O X O X O X X X 
6 O X O X O X X X 
7 O O X X X X X X 
8 . X X X O O O O 

[(8, 'A')]
Estoy en las acciones de min
  A B C D E F G H
1 O O O O O X X X 
2 O O O O X X X X 
3 O O O X O O X X 
4 O O X O O X X X 
5 O X O X O X X X 
6 O X O X O X X X 
7 O O X X X X X X 
8 O O O O O O O O 

Estoy en las acciones de min
  A B C D E F G H
1 O O O O O X X X 
2 O O O O X X X X 
3 O O O X O O X X 
4 O O X O X X X X 
5 O X O X X X X X 
6 O O X X X O X X 
7 O X O O O X O X 
8 . X . O . . X O 

[(8, 'C'), (8, 'E'), (8, 'F')]
Estoy en las acciones de max
  A B C D E F G H
1 O O O O O X X X 
2 O O O O X X X X 
3 O O O X O O X X 
4 O O X O X X X X 
5 O X O X X X X X 
6 O O X X X O X X 
7 O X X X O X O X 
8 . X X O . . X O 

[(8, 'A'), (8, 'E'), (8, 'F')]
Estoy en las acciones de min
  A B C D E F G H
1 O O O O O X X X 
2 O O O O X X X X 
3 O 

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-38-6e71b97a2295>", line 2, in <cell line: 2>
    min_max(initial_state, "MAX")
  File "<ipython-input-34-2b6093b82198>", line 37, in min_max
    v.append((min_value(successor), action))
  File "<ipython-input-34-2b6093b82198>", line 25, in min_value
    eval_max = max_value(successor) # evaluar el estado
  File "<ipython-input-34-2b6093b82198>", line 11, in max_value
    eval_min = min_value(successor) # evaluar el estado
  File "<ipython-input-34-2b6093b82198>", line 25, in min_value
    eval_max = max_value(successor) # evaluar el estado
  File "<ipython-input-34-2b6093b82198>", line 11, in max_value
    eval_min = min_value(successor) # evaluar el estado
  File "<ipython-input-34-2b6093b82198>", line 25, in min_value
    eval_max = max_value(successor) # evaluar el est

TypeError: ignored

In [ ]:
import copy

def jugando(state):
    print("Desea que la máquina inicie el juego? s/n")
    resp = input()

    if resp == "s":
        state_copia = copy.deepcopy(state)
        jugada = min_max(state_copia, "MAX")  # Asume que tienes la función min_max implementada
        state.realizar_movimiento(jugada[0], jugada[1], 'X')

    while not state.es_juego_terminado():
        state.mostrar_tablero()

        # Aquí deberías implementar la lógica para que el usuario haga su jugada
        # Podrías pedirle las coordenadas de su jugada y usar state.realizar_movimiento(fila, columna, 'O')
        # Recuerda verificar si el juego ha terminado después de cada jugada del usuario

        if not state.es_juego_terminado():
            state_copia = copy.deepcopy(state)
            jugada = min_max(state_copia, "MAX")  # Asume que tienes la función min_max implementada
            state.realizar_movimiento(jugada[0], jugada[1], 'X')

    # El juego ha terminado, puedes mostrar el tablero final y el resultado
    state.mostrar_tablero()
    ganador = state.obtener_ganador()
    if ganador == 'X':
        print("¡La máquina gana!")
    elif ganador == 'O':
        print("¡El usuario gana!")
    else:
        print("Empate.")


In [ ]:
initial_state = Estado()
jugando(initial_state)

Desea que la máquina inicie el juego? s/n


KeyboardInterrupt: ignored